<a href="https://colab.research.google.com/github/OmSatpute/AITrafficSystem/blob/main/AI_TrafficSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import time
from xml.etree import ElementTree as ET

# Load the AI model for vehicle detection (pre-trained model)
model = tf.keras.models.load_model('vehicle_detection_model.h5')

# Load the cars.xml file for vehicle classification (if needed)
car_classifier = cv2.CascadeClassifier('cars.xml')

def capture_images(camera_id=0):
    """Captures an image from the camera."""
    cap = cv2.VideoCapture(camera_id)
    ret, frame = cap.read()
    if ret:
        cap.release()
        return frame
    else:
        cap.release()
        raise Exception("Failed to capture image")

def process_image(image):
    """Processes the captured image for vehicle detection."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    vehicles = car_classifier.detectMultiScale(gray, 1.1, 1)
    return vehicles

def apply_ai_algorithm(image, vehicles):
    """Applies AI algorithm to classify and calculate vehicle density."""
    density = 0
    for (x, y, w, h) in vehicles:
        vehicle_img = image[y:y+h, x:x+w]
        resized_img = cv2.resize(vehicle_img, (128, 128)) / 255.0
        prediction = model.predict(np.expand_dims(resized_img, axis=0))
        if np.argmax(prediction) == 1:  # Assuming '1' indicates a vehicle
            density += 1
    return density

def generate_timing(density):
    """Generates traffic light timing based on vehicle density."""
    if density > 20:
        return 60  # Green light for 60 seconds
    elif density > 10:
        return 40  # Green light for 40 seconds
    else:
        return 20  # Green light for 20 seconds

def send_to_cloud(data):
    """Sends data to the cloud (Placeholder function)."""
    print("Sending data to the cloud:", data)

def main():
    while True:
        # Step 1: Capture image
        image = capture_images()

        # Step 2: Process the image
        vehicles = process_image(image)

        # Step 3: Apply AI algorithm
        density = apply_ai_algorithm(image, vehicles)

        # Step 4: Generate traffic light timing
        timing = generate_timing(density)

        # Step 5: Send data to the cloud
        data = {"density": density, "timing": timing}
        send_to_cloud(data)

        # Wait for a few seconds before the next iteration
        time.sleep(5)

if __name__ == "__main__":
    main()
